In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487-final-project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

['checkpoint_7.pth', 'checkpoint_no_na.pth', 'checkpoint_tuned.pth', 'checkpoint_tuned_1.pth', 'checkpoint_tuned_2.pth', 'credentials.pyc', 'credentials.py', '__pycache__', 'requirements.txt', 'preprocess.py', 'nb.ipynb', 'test.py', 'README.md', 'eval.py', 'scrape.py', 'naivebayes.py', 'checkpoint.pth', 'data', 'twitter.py', 'checkpoint_twitter.pth', 'checkpoint_twitter_2.pth', 'model.py', 'checkpoint_tuned_3.pth', 'checkpoint_tuned_8.pth', 'train_model.ipynb', 'train_tweet_model.ipynb']


In [3]:
!pip install transformers
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from google.colab import files

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

In [5]:
from model import train
from model import BertClassifier

In [6]:
np.random.seed(112)

df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'data/tweets_2.csv'))
df.dropna()
df.rename(columns={'label': 'type', 'tweet': 'article'}, inplace=True)
df_train, df_test = np.split(df.sample(frac=1, random_state=42), [int(.9*df.shape[0])])

In [7]:
num_epochs = 5
best_hyperparameters = {'learning_rate': 1e-05, 'hidden_size': 256, 'batch_size': 4}

clf = BertClassifier(hidden_size=best_hyperparameters['hidden_size'])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train(clf, df_train, df_test, best_hyperparameters['learning_rate'], num_epochs, best_hyperparameters['batch_size'])
torch.save(clf.state_dict(), os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_twitter_2.pth'))

files.download(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_twitter_2.pth'))

Using Cuda: True


100%|██████████| 2250/2250 [08:50<00:00,  4.24it/s]


Epochs: 1 | Train Loss:  0.143             | Train Accuracy:  0.717             | Val Loss:  0.102             | Val Accuracy:  0.815


100%|██████████| 2250/2250 [08:49<00:00,  4.25it/s]


Epochs: 2 | Train Loss:  0.060             | Train Accuracy:  0.903             | Val Loss:  0.099             | Val Accuracy:  0.835


...

Epochs: 4 | Train Loss:  0.009             | Train Accuracy:  0.989             | Val Loss:  0.132             | Val Accuracy:  0.838


100%|██████████| 2250/2250 [08:49<00:00,  4.25it/s]


Epochs: 5 | Train Loss:  0.007             | Train Accuracy:  0.989             | Val Loss:  0.138             | Val Accuracy:  0.831


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from model import Dataset
from eval import Evaluation


cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

model_test = BertClassifier(hidden_size=best_hyperparameters['hidden_size'])
state_dict = torch.load(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_twitter_2.pth'))
model_test.load_state_dict(state_dict)
model_test.to(device)

test_data = Dataset(df_test)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=best_hyperparameters['batch_size'], shuffle=True)

total_correct = 0
with torch.no_grad():
    preds = []
    labels = []
    for test_input, test_label in tqdm(test_dataloader):
        test_label = test_label.to(device)
        attention_mask = test_input['attention_mask'].to(device)
        input_id = test_input['input_ids'].squeeze(1)
        input_id = input_id.to(device)

        output = model_test(input_id, attention_mask)
        num_correct = (output.argmax(dim=1) == test_label).sum().item()
        preds.extend(output.argmax(dim=1).tolist())
        labels.extend(test_label.tolist())
        total_correct += num_correct

eval = Evaluation(preds, labels)
eval.all_metrics()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

100%|██████████| 250/250 [00:19<00:00, 12.77it/s]

Accuracy: 0.836
-----------------------
Macro
F1 score macro: 0.8358713231173239
Precision macro: 0.838146919907547
Recall macro: 0.8365559649691339
-----------------------
Micro
F1 score micro: 0.836
Precision micro: 0.836
Recall micro: 0.836


# Test Article model on tweets

In [11]:
from model import Dataset
from eval import Evaluation


cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

model_test = BertClassifier(hidden_size=best_hyperparameters['hidden_size'])
state_dict = torch.load(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_tuned_8.pth'))
model_test.load_state_dict(state_dict)
model_test.to(device)

test_data = Dataset(df_test)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=best_hyperparameters['batch_size'], shuffle=True)

total_correct = 0
with torch.no_grad():
    preds = []
    labels = []
    for test_input, test_label in tqdm(test_dataloader):
        test_label = test_label.to(device)
        attention_mask = test_input['attention_mask'].to(device)
        input_id = test_input['input_ids'].squeeze(1)
        input_id = input_id.to(device)

        output = model_test(input_id, attention_mask)
        num_correct = (output.argmax(dim=1) == test_label).sum().item()
        preds.extend(output.argmax(dim=1).tolist())
        labels.extend(test_label.tolist())
        total_correct += num_correct

eval = Evaluation(preds, labels)
eval.all_metrics()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 250/250 [00:19<00:00, 12.82it/s]

Accuracy: 0.505
-----------------------
Macro
F1 score macro: 0.31331128418507065
Precision macro: 0.3506457684319834
Recall macro: 0.3392624954491081
-----------------------
Micro
F1 score micro: 0.505
Precision micro: 0.505
Recall micro: 0.505



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
